In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pygame

In [3]:
import cv2
import pygame
import time

# Initialiser pygame pour le son
pygame.mixer.init()

# Charger le fichier audio pour l'alarme
alarm_sound = pygame.mixer.Sound(r"D:\Attention_Beep.wav")

# Charger les classificateurs Haar pour les visages et les yeux
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Démarrer la capture vidéo
cap = cv2.VideoCapture(0)

# Variables pour la gestion du temps
eyes_closed_start_time = None
alarm_played = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Détecter les visages dans l'image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Dessiner un rectangle autour du visage
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Région d'intérêt (ROI) pour les yeux dans le visage détecté
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        # Détecter les yeux dans le ROI du visage
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) == 0:
            # Si aucun œil n'est détecté, cela signifie que les yeux sont fermés
            cv2.putText(frame, "Eyes Closed", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            
            # Si c'est la première fois que les yeux sont fermés, commencer le timer
            if eyes_closed_start_time is None:
                eyes_closed_start_time = time.time()
            
            # Vérifier si les yeux sont fermés depuis plus de 2 secondes
            elif time.time() - eyes_closed_start_time >= 2:
                # Si plus de 2 secondes se sont écoulées, jouer l'alarme si elle n'a pas été jouée
                if not alarm_played:
                    alarm_sound.play()
                    alarm_played = True

        else:
            # Si des yeux sont détectés, cela signifie qu'ils sont ouverts
            cv2.putText(frame, "Eyes Open", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
            # Réinitialiser le timer lorsque les yeux sont ouverts
            eyes_closed_start_time = None
            alarm_played = False

        # Dessiner un rectangle autour des yeux détectés
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

    # Afficher la fenêtre avec les résultats
    cv2.imshow('Eye Detection', frame)

    # Quitter la boucle si 'q' est pressé
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer la capture et fermer les fenêtres
cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
